In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('data_cleaned.csv')
df.head()

,Airline,Source,Destination,Total_Stops,Price,journey_day,journey_month,dep_time_hour,dep_time_min,arrival_time_hour,arrival_time_min,Route 1,Route 2,Route 3,Route 4,Route 5,Duration_hours,Duration_mins
0,IndiGo,Banglore,New Delhi,0,3897,24,3,22,20,1,10,BLR,DEL,None,None,None,2,50
1,Air India,Kolkata,Banglore,2,7662,5,1,5,50,13,15,CCU,IXR,BBI,BLR,None,7,25
2,Jet Airways,Delhi,Cochin,2,13882,6,9,9,25,4,25,DEL,LKO,BOM,COK,None,19,0
3,IndiGo,Kolkata,Banglore,1,6218,5,12,18,5,23,30,CCU,NAG,BLR,None,None,5,25
4,IndiGo,Banglore,New Delhi,1,13302,3,1,16,50,21,35,BLR,NAG,DEL,None,None,4,45


In [ ]:
cat_col = [col for col in df.columns if df[col].dtype == 'O' ]

In [ ]:
cat_col

In [6]:
x = df.drop('Price' ,axis = 1)
y = df['Price']

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [ ]:
encoder = OneHotEncoder(sparse = False)

In [ ]:
encoder.fit(x[cat_col])

In [ ]:
step1 = ColumnTransformer([('step1' , OneHotEncoder(categories = encoder.categories_ ,sparse = False) , cat_col)])

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor

In [ ]:
# best model is  bag = BaggingRegressor( base_estimator = RandomForestRegressor() )

In [ ]:
model = BaggingRegressor( base_estimator = RandomForestRegressor() )

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
pipe = Pipeline([('step1' , step1) ,('step2' , model)])

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size = 0.2 , random_state = 123)

In [ ]:
pipe.fit(x_train,y_train)

In [ ]:
cols = x_train.columns

In [ ]:
for i in cols:
    print(f'{i} {df[i].unique()}')
    print('\n')

In [3]:
input_feature = ['Air India','Delhi' , 'Hyderabad' , 3 ,  4 , 1 , 9 , 20, 23,30 , 'DEL' , 'NAG' , 'BOM' , "None" , "None" , 7 ,0]

In [4]:
input_array = np.array([input_feature])

In [ ]:
input_array.shape

In [ ]:
x.columns

In [7]:
pred = pd.DataFrame(data = input_array , columns = x.columns)
pred

,Airline,Source,Destination,Total_Stops,journey_day,journey_month,dep_time_hour,dep_time_min,arrival_time_hour,arrival_time_min,Route 1,Route 2,Route 3,Route 4,Route 5,Duration_hours,Duration_mins
0,Air India,Delhi,Hyderabad,3,4,1,9,20,23,30,DEL,NAG,BOM,None,None,7,0


In [ ]:
pipe.predict(pred)

In [9]:
import pickle

In [ ]:
pickle.dump(pipe , open("model.pkl","wb"))

In [10]:
model = pickle.load(open("model.pkl" , "rb"))

In [11]:
model.predict(pred)

array([8880.53852668])